In [27]:
import pickle
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from emoji import UNICODE_EMOJI
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from textblob import Word
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve


In [28]:
df= pd.read_csv('C:/Users/91949/Desktop/ML/grpproj/1_2000_Labelled.csv')
comments = df
x=df['Comments']
y=df['Target']


In [29]:
df.head()

,Comments,Target
0,['I told my parents and they said that he was ...,0
1,['Sick man'],1
2,['Trump youre fired'],1
3,['Let Obama say some shit like this'],1
4,['Ive known my father my whole life REALLY Wow...,0


In [30]:

# brief clean up
def preprocess(comment):
    comment = comment.str.replace("\n", " ") # remove new lines 
    return comment

df['Comments'] = preprocess(df['Comments'])
df.head()

,Comments,Target
0,['I told my parents and they said that he was ...,0
1,['Sick man'],1
2,['Trump youre fired'],1
3,['Let Obama say some shit like this'],1
4,['Ive known my father my whole life REALLY Wow...,0


In [31]:
scores = []
analyzer = SentimentIntensityAnalyzer()
# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
for i in range(df['Comments'].shape[0]):
#print(analyser.polarity_scores(sentiments_pd['text'][i]))
    compound = analyzer.polarity_scores(df['Comments'][i])["compound"]
    pos = analyzer.polarity_scores(df['Comments'][i])["pos"]
    neu = analyzer.polarity_scores(df['Comments'][i])["neu"]
    neg = analyzer.polarity_scores(df['Comments'][i])["neg"]
    
    scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg
                 })
  

In [32]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score,how='left')


In [33]:
ypred=[]
for i in range(df['Comments'].shape[0]):
    if df["Positive"][i] >df["Negative"][i]:
        x=0
    else:
        x=1
    ypred.append(x)
df['ypred'] = ypred
df


,Comments,Target,Compound,Positive,Negative,ypred
0,['I told my parents and they said that he was ...,0,0.5423,0.241,0.000,0
1,['Sick man'],1,-0.5106,0.000,0.767,1
2,['Trump youre fired'],1,-0.5574,0.000,0.643,1
3,['Let Obama say some shit like this'],1,-0.2732,0.225,0.324,1
4,['Ive known my father my whole life REALLY Wow...,0,0.3455,0.227,0.160,0
...,...,...,...,...,...,...
2816,['I really cant believe people still support D...,1,0.4019,0.252,0.000,0
2817,['CNN u busted'],1,0.0000,0.000,0.000,1
2818,['this again CNN sure loves reposting old news'],1,0.7184,0.500,0.000,0
2819,['FUCKCNN'],1,0.0000,0.000,0.000,1


In [35]:
t=classification_report(y,ypred)
t

'              precision    recall  f1-score   support\n\n           0       0.50      0.39      0.44      1074\n           1       0.67      0.76      0.71      1747\n\n    accuracy                           0.62      2821\n   macro avg       0.58      0.57      0.57      2821\nweighted avg       0.60      0.62      0.61      2821\n'

In [36]:
k=confusion_matrix(y, ypred)
k

array([[ 418,  656],
       [ 420, 1327]], dtype=int64)

In [38]:
## Another method 
## credit: https://towardsdatascience.com/a-complete-exploratory-data-analysis-and-visualization-for-text-data-29fb1b96fb6a
comments['textblob_polarity'] = comments['Comments'].map(lambda text: TextBlob(text).sentiment.polarity)

comments['comment_len'] = comments['Comments'].astype(str).apply(len)
comments['word_count'] = comments['Comments'].apply(lambda x: len(str(x).split()))


In [39]:
# most positive comments
comments.sort_values(by=['textblob_polarity'], ascending=False)[['Comments', 'textblob_polarity']].head(10)

,Comments,textblob_polarity
2580,['Comrade Trump believes the republicon adage ...,1.000
155,['Barack Obama is the best father'],1.000
149,['the best bodyLOL'],1.000
586,['WeLl hEs nOt thE beSt wiTh WoRdS'],1.000
1519,['Ivanka is very beautiful'],1.000
339,['him and biden are the best americas got btw ...,1.000
2416,['Incest is best for trump brood'],1.000
1108,['Hes just very happy that hes created a beaut...,0.925
454,['One thing we can all agree his daughter is r...,0.850
1216,['So Beautiful '],0.850


In [40]:
# most negative
comments.sort_values(by=['textblob_polarity'], ascending=True)[['Comments', 'textblob_polarity']].head(10)

,Comments,textblob_polarity
1876,['TRUMP WHAT A NASTY PIG'],-1.0
2766,['Wonder if he thinks Tiffany is a dog What a ...,-1.0
827,['CNN is disgusting'],-1.0
1771,['Disgusting'],-1.0
115,['He is really disgusting '],-1.0
833,['Ewwww nasty hes nasty'],-1.0
2336,['Thats toooo disgusting and Nasty '],-1.0
282,['Disgusting'],-1.0
273,['absolutely disgusting'],-1.0
1544,['thats really fucking disgusting'],-1.0


In [54]:
ypred_2=[]
textblob_polarity = comments['Comments'].map(lambda text: TextBlob(text).sentiment.polarity)
comments['']=textblob_polarity


In [55]:
for i in range(comments['Comments'].shape[0]):
    if (comments["textblob_polarity"][i])>0:
        x=0
    else:
        x=1
    ypred_2.append(x)
comments['ypred'] = ypred_2

comments

,Comments,Target,,textblob_polarity,comment_len,word_count,ypred
0,['I told my parents and they said that he was ...,0,0.000000,0.000000,64,12,1
1,['Sick man'],1,-0.714286,-0.714286,12,2,1
2,['Trump youre fired'],1,0.000000,0.000000,21,3,1
3,['Let Obama say some shit like this'],1,-0.200000,-0.200000,37,7,1
4,['Ive known my father my whole life REALLY Wow...,0,-0.166667,-0.166667,75,15,1
...,...,...,...,...,...,...,...
2816,['I really cant believe people still support D...,1,0.200000,0.200000,59,9,0
2817,['CNN u busted'],1,0.000000,0.000000,16,3,1
2818,['this again CNN sure loves reposting old news'],1,0.300000,0.300000,48,8,0
2819,['FUCKCNN'],1,0.000000,0.000000,11,1,1


In [56]:
t2=classification_report(y,ypred_2)
t

'              precision    recall  f1-score   support\n\n           0       0.50      0.39      0.44      1074\n           1       0.67      0.76      0.71      1747\n\n    accuracy                           0.62      2821\n   macro avg       0.58      0.57      0.57      2821\nweighted avg       0.60      0.62      0.61      2821\n'

In [57]:
k=confusion_matrix(y, ypred_2)
k

array([[ 389,  685],
       [ 446, 1301]], dtype=int64)